# Natural orbitals

The [one-particle density](https://kthpanor.github.io/echem/docs/elec_struct/reduced_density.html#one-particle-density) and the associated spin-orbital density matrix, $D^\mathrm{SO}$, are related as follows

$$
n(\mathbf{r}) \equiv
\sum_{p,q} \psi^\dagger_p(\mathbf{r}) D_{pq}^\mathrm{SO} \psi_q(\mathbf{r})
$$

A nonzero contribution to the density requires the spin of both orbitals to be identical and the summation can therefore be separated by introducing the $\alpha$- and $\beta$-spin densities and instead run over molecular orbitals

\begin{align*}
n(\mathbf{r}) & =
\sum_{p,q} 
\Big(
\big[
\phi^\alpha_p(\mathbf{r})
\big]^\ast
D_{pq}^\alpha
\phi_q^\alpha(\mathbf{r})
+
\big[
\phi^\beta_p(\mathbf{r})
\big]^\ast
D_{pq}^\beta
\phi_q^\beta(\mathbf{r})
\Big)
\end{align*}

Natural orbitals is a single set of spatial orbitals in which the one-particle density is expanded

$$
n(\mathbf{r}) =
\sum_{p}
\big[
\phi_p^\mathrm{NO}(\mathbf{r})
\big]^\ast
D_{pp}^\mathrm{NO}
\phi_p^\mathrm{NO}(\mathbf{r})
$$

and the elements of the diagonal density matrix, $D_{pp}^\mathrm{NO}$, in this basis are referred to as occupation numbers.

## Restricted orbitals

In restricted calculations, we have a common set of MOs for the $\alpha$- and $\beta$-spin orbitals and can write

\begin{align*}
n(\mathbf{r}) & =
\sum_{p,q} 
\phi_p^\ast(\mathbf{r})
D_{pq}
\phi_q(\mathbf{r})
\\
D_{pq} & =
D_{pq}^\alpha + D_{pq}^\beta
\end{align*}

The density matrix is Hermitian and can be diagonalized by a unitary transformation. We get

\begin{align*}
\mathbf{D}^\mathrm{NO} & = 
\mathbf{U}^\dagger
\mathbf{D}
\mathbf{U} \\
\phi_p^\mathrm{NO}(\mathbf{r}) & =
\sum_q
\phi_q(\mathbf{r})
U_{qp}
\end{align*}

In restricted closed (RHF) and open-shell Hartree–Fock (ROHF), the natural orbitals equal the HF orbitals and the occupation numbers equal to 0, 1, and 2 for unoccuppied, singly occupied, and doubly occupied MOs, respectively.

For the case of a multi-configurational wave function, we study the oxygen molecule as an example.

In [ ]:
import multipsi as mtp
import numpy as np
import veloxchem as vlx

mol_str = """2

O   -0.6   0.0  0.0
O    0.6   0.0  0.0
"""
molecule = vlx.Molecule.read_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)
molecule.set_multiplicity(3)

rohf_drv = vlx.ScfRestrictedOpenDriver()
rohf_drv.ostream.mute()
rohf_results = rohf_drv.compute(molecule, basis)

In [ ]:
molecule.show()

In [ ]:
print(f" RHF energy: {rohf_drv.get_scf_energy() : 14.8f}")

The MOs are occupied by nine $\alpha$- and seven $\beta$-spin electrons.

In [ ]:
rohf_drv.molecular_orbitals.print_orbitals(molecule, basis)

**Main AO character of MOs**

---
Active orbitals (8 electrons)

- $3\sigma_u = p_z^1 + p_z^2$

- $1\pi_g = p_{x,y}^1 - p_{x,y}^2$

- $1\pi_u = p_{x,y}^1 + p_{x,y}^2$

- $3\sigma_g = p_z^1 - p_z^2$

---
Inactive orbitals (8 electrons)

- $2\sigma_u = 2s^1 - 2s^2$

- $2\sigma_g = 2s^1 + 2s^2$

- $1\sigma_u = 1s^1 - 1s^2$

- $1\sigma_g = 1s^1 + 1s^2$

---

In [ ]:
casscf_drv = mtp.McscfDriver()
orbital_space = mtp.OrbSpace(molecule, rohf_drv.mol_orbs)
orbital_space.cas(8, 6)

casscf_results = casscf_drv.compute(molecule, basis, orbital_space)

In [ ]:
print(f"CAS(8,6) energy: {casscf_results["energies"][0] : 12.8f}")

The wave function is predominantly a sum of two determinants with $\alpha$-spin occupation of the $\pi_u$ and $\pi_g$ molecular orbitals, respectively.

In [ ]:
casscf_results["ci_vectors"].print(0)

The occupation numbers are (i) equal to 2 for the four inactive orbitals, (ii) close to 2 for $3\sigma_g$ and $1\pi_u$, (iii) close to 1 for $1\pi_g$, and (iv) close to 0 for $3\sigma_u$.

In [ ]:
print(casscf_results["natural_occupations"][0])

## Unrestricted orbitals

In unrestricted calculations, we have separate sets of MOs for the $\alpha$- and $\beta$-spin orbitals. However, one set can be expressed in terms of the other according to

$$
\phi^\beta_q(\mathbf{r})
=
\sum_p \phi^\alpha_p(\mathbf{r}) S_{pq}^{\alpha\beta}
$$

where $S^{\alpha\beta}$ is the overlap between the two sets of orbitals that, in turn, can be expressed in terms of the overlap matrix in the AO basis

$$
S_{pq}^{\alpha\beta} =
\sum_{\delta,\gamma}
\big[c_{\delta p}^\alpha\big]^\ast c^\beta_{\gamma q}
S_{\delta \gamma}
$$

The one-particle density becomes

\begin{align*}
n(\mathbf{r}) & =
\sum_{p,q} 
\big[
\phi^\alpha_p(\mathbf{r})
\big]^\ast
D_{pq}
\phi_q^\alpha(\mathbf{r})
\\
D_{pq} & = D_{pq}^\alpha +
\sum_{r,s} 
\big[
S^{\alpha\beta}_{pr}
\big]^\ast
D_{rs}^\beta
S^{\alpha\beta}_{qs}
\end{align*}

and the natural orbitals and occupation numbers are obtained from a diagonalization of this total density matrix.

Let us consider the unrestricted Hartree–Fock state of the oxygen molecule.

In [ ]:
uhf_drv = vlx.ScfUnrestrictedDriver()
uhf_drv.ostream.mute()
uhf_results = uhf_drv.compute(molecule, basis)

In [ ]:
print(f"UDFT energy: {uhf_drv.get_scf_energy() : 14.8f}")

In [ ]:
norb = basis.get_dimensions_of_basis()
n_alpha_electrons = 9
n_beta_electrons = 7

S = uhf_results["S"]
C_alpha = uhf_results["C_alpha"]
C_beta = uhf_results["C_beta"]

D_alpha = np.zeros((norb, norb))
for i in range(n_alpha_electrons):
    D_alpha[i, i] = 1

D_beta = np.zeros((norb, norb))
for i in range(n_beta_electrons):
    D_beta[i, i] = 1

In [ ]:
S_ab = np.einsum("ap, bq, ab -> pq", C_alpha, C_beta, S)
D = D_alpha + np.einsum("pr, qs, rs -> pq", S_ab, S_ab, D_beta)

occ_numbers, U = np.linalg.eigh(D)

In [ ]:
print("Orbital  Occ. number")
for orb, occ in enumerate(list(occ_numbers)[::-1]):
    if occ > 1e-3:
        print(f"{orb + 1 : >2} {occ : 12.3f}")

We can also get the natural orbitals and occupation numbers from the `natural_orbitals` method in VeloxChem.

In [ ]:
natural_orbs = uhf_drv.natural_orbitals(uhf_results)
natural_orbs.print_orbitals(molecule, basis)